# Make clean data

In [27]:
import pandas as pd

bubbleList = []
countingList = []
mergeList = []
quickList = []
readCSVList = []
pcList = ["6", "16", "21", "23"]

for i in range(len(pcList)):
    bubbleList.append(pd.read_csv("Results"+pcList[i]+"/bubbleSort-PC"+pcList[i]+".csv"))
    bubbleList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

for i in range(len(pcList)):
    countingList.append(pd.read_csv("Results"+pcList[i]+"/countingSort-PC"+pcList[i]+".csv"))
    countingList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]
    
for i in range(len(pcList)):
    mergeList.append(pd.read_csv("Results"+pcList[i]+"/mergeSort-PC"+pcList[i]+".csv"))
    mergeList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

for i in range(len(pcList)):
    quickList.append(pd.read_csv("Results"+pcList[i]+"/quickSort-PC"+pcList[i]+".csv"))
    quickList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

for i in range(len(pcList)):
    readCSVList.append(pd.read_csv("Results"+pcList[i]+"/readCSV-PC"+pcList[i]+".csv"))
    readCSVList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]



In [28]:
def removeNonsense(df):
    df = df[df["PKG (J)"]>=0]
    df = df[df["CPU (J)"]>=0]
    df = df[df["Time (ms)"]>=0]
    return df

In [29]:
def makeinputColumn(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[0])

    df["Input case"] = l

def makeSizecolumn(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[1]))

    df["Size"] = l

In [30]:
def makeinputColumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[1].lower())
    df["Input case"] = l

def makeSizecolumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[-1]))

    df["Size"] = l

In [31]:
def removeOutliers(df, type_, size, col):
    q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
    q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
    IQR = q3 - q1 
    LF = q1 - 1.5 * IQR
    UF = q3 + 1.5 * IQR

    df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
    df2 = df1[df1["Input case"] == type_][df1["Size"]==size][df1[col] >= LF]
    return df2[col]


In [32]:
def CleanDataframe(df, DataOrBase):
    if df["Size"].iloc[-1] < 1000000:
        sizes = [25000,50000,75000,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000]
    else:
        sizes = [25000,50000,75000,100000,200000,300000,400000,500000]

    if DataOrBase == "data":
        cases = ["best", "random", "worst"]
    else:
        cases = ["random", "countingrandom", "countingbest", "countingworst", "mergeworst", "quickbest", "reversesorted", "sorted"]

    cleanList = []
    for case in cases:
        if case != "random" and case != "countingrandom":
            for size in sizes:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        else:
            for i in range(1,11):
                for size in sizes:
                    cleanPortion = pd.DataFrame({"PKG (J)":[]})
                    for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                        cleanPortion[col] = removeOutliers(df, case+str(i), size, col)
                
                    cleanPortion["Input case"] = [case+str(i) for _ in range(len(cleanPortion))]
                    cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]
                    cleanList.append(cleanPortion)
    cleanBubble = pd.concat(cleanList)
    return cleanBubble

In [33]:
def prepareDataframe(df, DataOrBase):
    df = removeNonsense(df)

    if DataOrBase == "data":
        makeinputColumn(df)
        makeSizecolumn(df)
    else:
        makeinputColumn_readCSV(df)
        makeSizecolumn_readCSV(df)    

    return df


In [34]:
def CleanCSV (df, DataOrBase, name, pcNO):
    ready = prepareDataframe(df, DataOrBase)
    clean = CleanDataframe(ready, DataOrBase).dropna().reset_index(drop=True)
    clean.to_csv("CleanData\\"+name+"Clean-PC"+pcNO+".csv", sep=",", index=False)

In [35]:
# for i in range(len(bubbleList)):
#     CleanCSV(bubbleList[i], "data","Bubble",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(countingList[i], "data","Counting",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(mergeList[i], "data","Merge",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(quickList[i], "data","Quick",pcList[i])

for i in range(len(bubbleList)):
    CleanCSV(readCSVList[i], "base","readCSV",pcList[i])

C:\Users\langk\AppData\Local\Temp\ipykernel_2784\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_2784\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
C:\Users\langk\AppData\Local\Temp\ipykernel_2784\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_2784\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_2784\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match DataF